In [ ]:
#this file computes the number of people using the metro between timestamps

In [1]:
import pandas as pd
from datetime import datetime,timedelta
mta_data = pd.read_pickle('mta_data.pickle')

In [ ]:
#reset index and drop old indices to remove duplicate indexes and fix cols
mta_data = mta_data.reset_index().drop('index',axis=1)
mta_data.columns = [col.strip() for col in mta_data.columns]


# convert dates to datetime, create col; convert that to day of week, create col; convert that to weekday T/F, create col
mta_data['weekday'] = pd.to_datetime(mta_data.DATE,format='%m/%d/%Y')
mta_data['weekday_n'] = mta_data.weekday.dt.dayofweek
mta_data['weekday_tf'] = mta_data.weekday_n<5

#create columns for entries since last time
mta_data['roll_ent'] = mta_data.groupby(['weekday'])['ENTRIES'].diff(periods = 1)
mta_data['roll_ex'] = mta_data.groupby(['weekday'])['EXITS'].diff(periods = 1)
mta_data.fillna(0,inplace=True)

#find mean of data by station and whether weekday or not
mean_entry = mta_data.groupby(['STATION','weekday_tf']).mean()
#mean total traffic, then only that
mean_entry['through'] = mean_entry.ENTRIES + mean_entry.EXITS
mean_entry.drop(labels = ['ENTRIES','EXITS','weekday_n'],axis=1,inplace=True)
#shift table so that each station's through is in cols T and F, then subtract
mean_entry.reset_index()
piv_me=mean_entry.pivot_table(index='STATION',columns='weekday_tf')
weekdaymean = piv_me['through']['weekday_tf'==False]
weekendmean = piv_me['through']['weekday_tf'==True]
weekdaymean-weekendmean

In [3]:
def find_growth(dataframe,column) :
    at_time = []
    at_time.append(0)
    for i in range(1,len(dataframe)) :
        if dataframe.iloc[i]['weekday']==dataframe.iloc[i-1]['weekday'] :
            at_time.append(dataframe.iloc[i][column]-dataframe.iloc[i-1][column])
        else :
            at_time.append(0)
    return pd.Series(at_time)

In [4]:
mta_data.columns = [col.strip() for col in mta_data.columns]

In [7]:
mta_data['roll_ent'] = mta_data.groupby(['weekday'])['ENTRIES'].diff(periods = 1)
mta_data['roll_ex'] = mta_data.groupby(['weekday'])['EXITS'].diff(periods = 1)
mta_data.fillna(0,inplace=True)

In [8]:
mean_entry = mta_data.groupby(['STATION','weekday_tf']).mean()
mean_entry['through'] = mean_entry.roll_ent + mean_entry.roll_ex
mean_entry.drop(labels = ['ENTRIES','EXITS','weekday_n','roll_ent','roll_ex'],axis=1,inplace=True)
mean_entry.reset_index()
piv_me=mean_entry.pivot_table(index='STATION',columns='weekday_tf')

weekdaymean = piv_me['through']['weekday_tf'==False].map(int)
weekendmean = piv_me['through']['weekday_tf'==True].map(int)
piv_me.head(5)
mta_data[mta_data.roll_ent<0]
mta_data.iloc[40:50]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,weekday,weekday_n,weekday_tf,roll_ent,roll_ex
40,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,08:10:05,REGULAR,5571173,1880736,2016-03-04,4,True,37.0,79.0
41,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,11:00:00,REGULAR,5571313,1881031,2016-03-04,4,True,140.0,295.0
42,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,15:00:00,REGULAR,5571587,1881113,2016-03-04,4,True,274.0,82.0
43,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,19:00:00,REGULAR,5572521,1881206,2016-03-04,4,True,934.0,93.0
44,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,23:00:00,REGULAR,5572864,1881239,2016-03-04,4,True,343.0,33.0
45,A002,R051,02-00-01,59 ST,NQR456,BMT,02/27/2016,03:00:00,REGULAR,5114878,1116740,2016-02-27,5,False,-448297.0,-761122.0
46,A002,R051,02-00-01,59 ST,NQR456,BMT,02/27/2016,07:00:00,REGULAR,5114889,1116760,2016-02-27,5,False,11.0,20.0
47,A002,R051,02-00-01,59 ST,NQR456,BMT,02/27/2016,11:00:00,REGULAR,5114945,1116819,2016-02-27,5,False,56.0,59.0
48,A002,R051,02-00-01,59 ST,NQR456,BMT,02/27/2016,15:00:00,REGULAR,5115178,1116872,2016-02-27,5,False,233.0,53.0
49,A002,R051,02-00-01,59 ST,NQR456,BMT,02/27/2016,19:00:00,REGULAR,5115572,1116929,2016-02-27,5,False,394.0,57.0


In [6]:
from datetime import datetime,timedelta
#reset index and drop old indices to remove duplicate indexes and fix cols
mta_data = mta_data.reset_index().drop('index',axis=1)
mta_data.columns = [col.strip() for col in mta_data.columns]


# convert dates to datetime, create col; convert that to day of week, create col; convert that to weekday T/F, create col
mta_data['weekday'] = pd.to_datetime(mta_data.DATE,format='%m/%d/%Y')
mta_data['weekday_n'] = mta_data.weekday.dt.dayofweek
mta_data['weekday_tf'] = mta_data.weekday_n<5

#create columns for entries since last time
mta_data['roll_ent'] = mta_data.groupby(['weekday'])['ENTRIES'].diff(periods = 1)
mta_data['roll_ex'] = mta_data.groupby(['weekday'])['EXITS'].diff(periods = 1)
mta_data.fillna(0,inplace=True)

#find mean of data by station and whether weekday or not
mean_entry = mta_data.groupby(['STATION','weekday_tf']).mean()
#mean total traffic, then only that
mean_entry['through'] = mean_entry.ENTRIES + mean_entry.EXITS
mean_entry.drop(labels = ['ENTRIES','EXITS','weekday_n'],axis=1,inplace=True)
#shift table so that each station's through is in cols T and F, then subtract
mean_entry.reset_index()
piv_me=mean_entry.pivot_table(index='STATION',columns='weekday_tf')
weekdaymean = piv_me['through']['weekday_tf'==False]
weekendmean = piv_me['through']['weekday_tf'==True]
weekdaymean-weekendmean

STATION
1 AV               0.0
103 ST             0.0
103 ST-CORONA      0.0
104 ST             0.0
110 ST             0.0
111 ST             0.0
116 ST             0.0
116 ST-COLUMBIA    0.0
121 ST             0.0
125 ST             0.0
135 ST             0.0
137 ST CITY COL    0.0
138/GRAND CONC     0.0
14 ST              0.0
14 ST-UNION SQ     0.0
145 ST             0.0
149/GRAND CONC     0.0
14TH STREET        0.0
15 ST-PROSPECT     0.0
155 ST             0.0
157 ST             0.0
161/YANKEE STAD    0.0
163 ST-AMSTERDM    0.0
167 ST             0.0
168 ST             0.0
169 ST             0.0
170 ST             0.0
174 ST             0.0
174-175 STS        0.0
175 ST             0.0
                  ... 
SUTPHIN-ARCHER     0.0
SUTTER AV          0.0
SUTTER AV-RUTLD    0.0
THIRTY ST          0.0
THIRTY THIRD ST    0.0
TIMES SQ-42 ST     0.0
TOMPKINSVILLE      0.0
TREMONT AV         0.0
TWENTY THIRD ST    0.0
UNION ST           0.0
UTICA AV           0.0
V.CORTLANDT PK     0.0
VAN

ENTRIES         EXITS      roll_ent  \
STATION         weekday_tf                                             
1 AV            False       2.020607e+08  2.062424e+08  9.691624e+06   
                True        2.040983e+08  2.051857e+08  9.851194e+06   
103 ST          False       1.000311e+07  3.956265e+07  8.053504e+04   
                True        9.986905e+06  3.948217e+07  8.157317e+04   
103 ST-CORONA   False       7.304913e+06  7.120379e+06  6.732629e+04   
                True        7.309255e+06  7.122326e+06  6.618859e+04   
104 ST          False       5.016144e+08  3.776540e+08  3.146720e+07   
                True        5.011722e+08  3.779363e+08  3.141100e+07   
110 ST          False       5.090116e+06  5.019973e+06  1.284646e+05   
                True        5.074642e+06  5.073581e+06  1.258125e+05   
111 ST          False       7.607974e+06  4.018397e+06 -1.779930e+07   
                True        7.594394e+06  4.007859e+06 -1.759021e+07   
116 ST          False       7.823318e+07  9.461871e+07 -4.500716e+05   
                True        7.766117e+07  9.391015e+07 -4.469961e+05   
116 ST-COLUMBIA False       2.026473e+08  4.316395e+07  4.345355e+05   
                True        2.027273e+08  4.318215e+07  4.343753e+05   
121 ST          False       4.191004e+06  1.393347e+06  2.304212e+05   
                True        3.750245e+06  1.397848e+06  1.881604e+05   
125 ST          False       1.398908e+08  7.595744e+07  4.304900e+06   
                True        1.399821e+08  7.622213e+07  4.387332e+06   
135 ST          False       1.069232e+07  2.054145e+06 -8.348077e+06   
                True        1.059490e+07  2.045857e+06 -8.475403e+06   
137 ST CITY COL False       3.813223e+07  8.931094e+07  2.222211e+04   
                True        3.799949e+07  8.889334e+07  2.214909e+04   
138/GRAND CONC  False       3.323614e+08  3.209268e+08  5.164935e+07   
                True        3.322929e+08  3.208627e+08  5.455524e+07   
14 ST           False       8.130844e+07  6.126274e+07  4.542543e+05   
                True        8.102053e+07  6.100759e+07  4.518603e+05   
14 ST-UNION SQ  False       2.904172e+07  9.666249e+06  9.695443e+04   
                True        2.920734e+07  9.685967e+06  9.438312e+04   
...                                  ...           ...           ...   
W 4 ST-WASH SQ  False       2.559512e+07  2.058137e+07 -9.047333e+05   
                True        2.559794e+07  2.049934e+07 -8.936274e+05   
W 8 ST-AQUARIUM False       1.458888e+06  1.564867e+06  9.212512e+03   
                True        1.458782e+06  1.562631e+06  9.135627e+03   
WAKEFIELD/241   False       3.650336e+06  3.771596e+06 -6.066922e+04   
                True        3.653341e+06  3.774581e+06 -6.095675e+04   
WALL ST         False       1.312886e+07  7.705290e+06 -3.274815e+05   
                True        1.310879e+07  7.710032e+06 -3.284271e+05   
WEST FARMS SQ   False       4.875412e+07  7.969326e+06 -1.276221e+07   
                True        4.871473e+07  7.963972e+06 -1.259753e+07   
WESTCHESTER SQ  False       8.549785e+06  7.906155e+06  6.080621e+05   
                True        8.554282e+06  7.910044e+06  5.977746e+05   
WHITEHALL S-FRY False       1.801077e+07  4.435878e+06 -1.991525e+04   
                True        1.802584e+07  4.436438e+06 -1.971866e+04   
WHITLOCK AV     False       2.480095e+07  2.656024e+06  1.231794e+06   
                True        2.486275e+07  2.660124e+06  1.220637e+06   
WILSON AV       False       3.523595e+06  3.167168e+06 -6.516565e+04   
                True        3.480290e+06  3.128397e+06 -6.904941e+04   
WINTHROP ST     False       6.768479e+06  1.371838e+06 -1.024368e+05   
                True        6.779713e+06  1.372778e+06 -1.016531e+05   
WOODHAVEN BLVD  False       7.244205e+06  3.851234e+06 -6.535366e+04   
                True        7.186852e+06  3.894738e+06 -6.476527e+04   
WOODLAWN        False       6.026216e+06  2.379303e+06  2.001329e+05   
 

In [59]:
mta_data['roll_ent'] = mta_data.groupby(['weekday'])['ENTRIES'].diff(periods = 1)
mta_data['roll_ex'] = mta_data.groupby(['weekday'])['EXITS'].diff(periods = 1)

In [60]:
mta_data.head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,roll_ent,weekday,weekday_n,weekday_tf,roll_ex
0,A002,R051,02-00-00,59 ST,NQR456,BMT,02/27/2016,03:00:00,REGULAR,5562172,1877574,NaN,2016-02-27,5,False,NaN
1,A002,R051,02-00-00,59 ST,NQR456,BMT,02/27/2016,07:00:00,REGULAR,5562184,1877586,12.0,2016-02-27,5,False,12.0
2,A002,R051,02-00-00,59 ST,NQR456,BMT,02/27/2016,11:00:00,REGULAR,5562267,1877691,83.0,2016-02-27,5,False,105.0
3,A002,R051,02-00-00,59 ST,NQR456,BMT,02/27/2016,15:00:00,REGULAR,5562516,1877745,249.0,2016-02-27,5,False,54.0
4,A002,R051,02-00-00,59 ST,NQR456,BMT,02/27/2016,19:00:00,REGULAR,5562920,1877821,404.0,2016-02-27,5,False,76.0
5,A002,R051,02-00-00,59 ST,NQR456,BMT,02/27/2016,23:00:00,REGULAR,5563175,1877862,255.0,2016-02-27,5,False,41.0
6,A002,R051,02-00-00,59 ST,NQR456,BMT,02/28/2016,03:00:00,REGULAR,5563262,1877882,NaN,2016-02-28,6,False,NaN
7,A002,R051,02-00-00,59 ST,NQR456,BMT,02/28/2016,07:00:00,REGULAR,5563273,1877892,11.0,2016-02-28,6,False,10.0
8,A002,R051,02-00-00,59 ST,NQR456,BMT,02/28/2016,11:00:00,REGULAR,5563346,1877954,73.0,2016-02-28,6,False,62.0
9,A002,R051,02-00-00,59 ST,NQR456,BMT,02/28/2016,15:00:00,REGULAR,5563545,1878026,199.0,2016-02-28,6,False,72.0


In [58]:
mta_data.iloc[40:50]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,roll_ent,weekday,weekday_n,weekday_tf,roll_ex
40,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,08:10:05,REGULAR,5571173,1880736,37.0,2016-03-04,4,True,79.0
41,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,11:00:00,REGULAR,5571313,1881031,140.0,2016-03-04,4,True,295.0
42,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,15:00:00,REGULAR,5571587,1881113,274.0,2016-03-04,4,True,82.0
43,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,19:00:00,REGULAR,5572521,1881206,934.0,2016-03-04,4,True,93.0
44,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,23:00:00,REGULAR,5572864,1881239,343.0,2016-03-04,4,True,33.0
45,A002,R051,02-00-01,59 ST,NQR456,BMT,02/27/2016,03:00:00,REGULAR,5114878,1116740,-448297.0,2016-02-27,5,False,-761122.0
46,A002,R051,02-00-01,59 ST,NQR456,BMT,02/27/2016,07:00:00,REGULAR,5114889,1116760,11.0,2016-02-27,5,False,20.0
47,A002,R051,02-00-01,59 ST,NQR456,BMT,02/27/2016,11:00:00,REGULAR,5114945,1116819,56.0,2016-02-27,5,False,59.0
48,A002,R051,02-00-01,59 ST,NQR456,BMT,02/27/2016,15:00:00,REGULAR,5115178,1116872,233.0,2016-02-27,5,False,53.0
49,A002,R051,02-00-01,59 ST,NQR456,BMT,02/27/2016,19:00:00,REGULAR,5115572,1116929,394.0,2016-02-27,5,False,57.0


In [61]:
mta_data[mta_data.roll_ent<0]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,roll_ent,weekday,weekday_n,weekday_tf,roll_ex
45,A002,R051,02-00-01,59 ST,NQR456,BMT,02/27/2016,03:00:00,REGULAR,5114878,1116740,-448297.0,2016-02-27,5,False,-761122.0
51,A002,R051,02-00-01,59 ST,NQR456,BMT,02/28/2016,03:00:00,REGULAR,5115864,1116969,-448146.0,2016-02-28,6,False,-761147.0
57,A002,R051,02-00-01,59 ST,NQR456,BMT,02/29/2016,03:00:00,REGULAR,5116530,1117111,-449245.0,2016-02-29,0,True,-761609.0
64,A002,R051,02-00-01,59 ST,NQR456,BMT,03/01/2016,03:00:00,REGULAR,5117887,1117455,-449649.0,2016-03-01,1,True,-761916.0
71,A002,R051,02-00-01,59 ST,NQR456,BMT,03/02/2016,03:00:00,REGULAR,5119371,1117783,-449897.0,2016-03-02,2,True,-762217.0
77,A002,R051,02-00-01,59 ST,NQR456,BMT,03/03/2016,03:00:00,REGULAR,5120845,1118110,-450210.0,2016-03-03,3,True,-762504.0
83,A002,R051,02-00-01,59 ST,NQR456,BMT,03/04/2016,03:00:00,REGULAR,5122391,1118478,-450473.0,2016-03-04,4,True,-762761.0
90,A002,R051,02-03-00,59 ST,NQR456,BMT,02/27/2016,03:00:00,REGULAR,654919,2467014,-4460877.0,2016-02-27,5,False,1350052.0
96,A002,R051,02-03-00,59 ST,NQR456,BMT,02/28/2016,03:00:00,REGULAR,655360,2468576,-4461149.0,2016-02-28,6,False,1351469.0
102,A002,R051,02-03-00,59 ST,NQR456,BMT,02/29/2016,03:00:00,REGULAR,655634,2469681,-4462238.0,2016-02-29,0,True,1352229.0
